<a href="https://colab.research.google.com/github/msiplab/EicEngLabIV/blob/master/example02_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実験Ⅳ－２　時系列データの分析

1. サンプルデータの読み込み
1. 平均 $\mu$ と分散 $\gamma_0$ の推定
1. 自己共分散関数 $\gamma_k$ と自己相関関数 $\rho_k$ の推定

新潟大学工学部工学科　電子情報通信プログラム 

## 準備

In [ ]:
#!pip install bs4
!pip install japanize-matplotlib
%matplotlib inline

import warnings
warnings.simplefilter('ignore') #警告を無視（コメントアウト推奨）

from urllib import request
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import pandas.tseries.offsets as offsets
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib #日本語化matplotlib
sns.set(font="IPAexGothic") #日本語フォント設定

## Webからの水位データの読み込み

新潟県河川防災情報システムより
http://doboku-bousai.pref.niigata.jp/kasen/index.html

河川ライブ映像 http://www.hrr.mlit.go.jp/shinano/live/map1.html# （小千谷市11番目）

以下のPythonライブラリを利用
- urllib
- bs4


In [ ]:
# Web スクレイピングかファイル入力かを選択
isWebScraping = False # Web スクレイピングを行う場合は True に設定

# 解析期間の設定
dts = '2022032501' # 開始日時
dte = '2022123123' # 終了日時

# タイムスタンプの設定
dts00 = pd.to_datetime(dts+'00').tz_localize('Asia/Tokyo')
dte00 = pd.to_datetime(dte+'00').tz_localize('Asia/Tokyo')

# ファイルをアップロードする
from google.colab import files
uploaded_file = files.upload()
filename = next(iter(uploaded_file))
print(filename)

#観測データのファイル
#filename = './data/practice02_01.csv'

if not isWebScraping:
    # ファイルからの水位情報抽出
    # CSVファイルの読み込み
    df_full = pd.read_csv(filename)
    df_full['t'] = pd.to_datetime(df_full['t'])    
    strtitle = filename
else:
    # Web スクレイピングを行う場合の設定
    # 取得したい水位観測所の情報を指定する
    # 水位観測所IDは、下記URLから取得可能
    # http://doboku-bousai.pref.niigata.jp/kasen/servlet/bousaiweb.servletBousaiGraph?ga=4&gk=0&gn=0&gl=0&gw=0&go=0&omp=0&opn=0&spn=0&tvm=0&tsw=0&sv=3&dk=2&mp=0&no=0&fn=0&pg=6&sitept=0&unq=12062215494&nwg=0&tmg={0}&sn={1}&wsl=3&wl=1&rg=1&sy=gra_river&psn=0&=&nw=0&tm={0}&logflg=0

    # 水位観測所IDの設定
    loc = '260' # 

    # 観測間隔の設定
    interval = 1 # hour

    # Webからの水位情報抽出
    td = dte00 - dts00
    durationinhours = int(td.total_seconds()/(3600))
    t = [ [] for idx in range(durationinhours) ]
    y = [ 0.0 for idx in range(durationinhours) ]
    idt = dts00
    idx = 0
    while idt < dte00:
        # 水位抽出日時の設定
        strdt = idt.strftime('%Y%m%d%H%M')
        ts = pd.to_datetime(idt) #.tz_localize('Asia/Tokyo')

        # URLの設定
        url = 'http://doboku-bousai.pref.niigata.jp/kasen/servlet/bousaiweb.servletBousaiGraph?ga=4&gk=0&gn=0&gl=0&gw=0&go=0&omp=0&opn=0&spn=0&tvm=0&tsw=0&sv=3&dk=2&mp=0&no=0&fn=0&pg=6&sitept=0&unq=12062215494&nwg=0&tmg={0}&sn={1}&wsl=3&wl=1&rg=1&sy=gra_river&psn=0&=&nw=0&tm={0}&logflg=0'.format(strdt,loc)

        # 指定した時刻の水位情報抽出
        response = request.urlopen(url)
        soup = BeautifulSoup(response)
        response.close()
        if idt == dts00:
            strtitle = soup.title.text.strip()
        #print(soup.find('td', class_='tableHeaderCast').text.strip())
        strwaterlevel = soup.find('td', class_='tableHeaderItemCen').text.strip().replace('m','')
        try:
            waterlevel = float(strwaterlevel)
        except ValueError as ve:
            waterlevel = np.nan

        # リストへのデータ登録
        t[idx] = ts
        y[idx] = waterlevel

        # 時間を更新
        idx += 1
        idt += offsets.Hour(interval)


## 時系列データをpandas.DataFrameオブジェクトに変換

pandas ライブラリの DataFrame オブジェクトに変換

- t: 時刻
- y: データの値


In [ ]:
if not isWebScraping:
    # Localize the timestamps to match the timezone of the 't' column
    df_timeseries = df_full[(df_full.t>=dts00) & (df_full.t<=dte00)].copy()
    df_timeseries = df_timeseries.reset_index()
else:
    df_timeseries = pd.DataFrame({'t': t, 'y': y})
display(df_timeseries)

## 時系列データをプロット

- y: 原系列
- y_fillna: 欠損個所を前の値で補間


In [ ]:
df_timeseries['y_fillna'] = df_timeseries['y'].fillna(method='ffill').astype(float)
ax = df_timeseries.plot(x='t', y=['y', 'y_fillna'], figsize=(16,4), title=strtitle)

ヒストグラムをプロット

In [ ]:
ax = df_timeseries.hist('y')
ax[0][0].set_title(strtitle)

## 平均 $\mu$ と分散 $\gamma_0$ の推定

- mean(): 標本平均を計算
- var(): 標本分散を計算

pandas.DataFrame オブジェクトのvar() メソッドのオプションで 

- ddof = 0 とすると N で割る最尤推定
- ddof = 1 とすると (N-1)で割る不偏推定 ※デフォルト

In [ ]:
y_series = df_timeseries['y']
mu = y_series.mean()
gamma0 = y_series.var(ddof=1) 
print('平均 = {:f}, 分散 = {:f}'.format(mu,gamma0))

## 自己共分散関数 $\gamma_k$ と自己相関関数 $\rho_k$ の推定

y_series は pandas.Series のオブジェクトとなっている。
pandas.Seriesオブジェクトは以下のメソッドを備える。

- cov() は共分散の不偏推定を計算
- corr() は相関の不偏推定を計算

※不偏推定から最尤推定への切替はできない


In [ ]:
nlags = 40
for lag in range(nlags+1):
    gammak = y_series.cov(y_series.shift(lag))
    rhok = y_series.corr(y_series.shift(lag))
    print('γ{0:d} = {1:f}, ρ{0:d} = {2:f}'.format(lag,gammak,rhok))

## 時系列解析ライブラリ

時系列解析には，statsmodelsライブラリも利用できる。

- acovf() は自己共分散関数を計算
- acf() は自己相関関数を計算

各メソッドのオプション unbiased = True で不偏推定，Falseで最尤推定となる．



In [ ]:
import statsmodels.api as sm
from statsmodels.tsa import stattools

#欠損値がある場合は nan が含まれる．
#欠損個所を補間したデータを使わない場合は以下の行をコメントアウトする
y_series = df_timeseries['y_fillna']

print('自己共分散関数')
y_acovf = stattools.acovf(y_series,unbiased=True)
display(y_acovf)
print('自己相関関数')
y_acf = stattools.acf(y_series,adjusted=True)
display(y_acf)


## 原系列のコレログラムの表示

自己相関のグラフをコレログラムとよぶ。コレログラムは次のコマンドで表示できる。

In [ ]:
fig = sm.graphics.tsa.plot_acf(y_series,lags=nlags)

## 階差系列の分析

ひとつ前の値との差分をとった系列を階差系列とよぶ。

$\Delta y_n = y_{n} - y_{n-1}$


In [ ]:
# 欠損データの補間を行わない場合
#df_timeseries['y_diff'] = df_timeseries['y'].diff()
# 欠損データを補間を行う場合
df_timeseries['y_diff'] = df_timeseries['y_fillna'].diff()
display(df_timeseries)
df_timeseries.plot(x='t', y='y_diff', figsize=(16,4), title='{}の階差系列'.format(strtitle))



ヒストグラムをプロット


In [ ]:
ax = df_timeseries.hist('y_diff')
ax[0][0].set_title(strtitle+'（階差）')

原系列 $y_n$ と階差系列 $\Delta y_n$ のCSVへの出力

In [ ]:
#df_timeseries.to_csv('./data/sample02_03.csv',index=False)